### 1.기본 손가락 추적

In [1]:
import cv2
import mediapipe as mp

# MediaPipe Hands 모듈 초기화 - 미리 설정후 시작해보기
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=2,               # 동시에 추적할 최대 손의 수
    min_detection_confidence=0.5,  # 손 검출을 위한 최소 신뢰도
    min_tracking_confidence=0.5)   # 손 추적을 위한 최소 신뢰도
mp_drawing = mp.solutions.drawing_utils  # 그리기 함수 초기화

# 웹캠 설정
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        continue

    # BGR 이미지를 RGB로 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 이미지 반전
    image = cv2.flip(image, 1)

    # 성능을 위해 이미지 쓰기 불가능으로 설정
    # MediaPipe를 통해 손 검출
    results = hands.process(image)

    # 이미지를 RGB에서 BGR로 변환하여 그리기 작업 준비
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # 검출된 손의 랜드마크를 이미지에 그림
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2))

    # 결과 이미지 표시
    cv2.imshow('Hand Tracking', image)

    # 'q'를 눌러 종료
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import mediapipe as mp
import cv2

# MediaPipe Pose 설정
pose = mp.solutions.pose.Pose(
    static_image_mode=False, # 입력 이미지가 정적 이미지인지, 비디오 스트림 프레임인지 설정(False인 경우 비디오 스트림으로 처리)
    model_complexity=1, # 0,1,2 의 순으로 정확도가 올라가는 대신 속도가 느림
    smooth_landmarks=True, # 프레임간의 랜드 마크 위치를 부드럽게 처리할 지 여부 결정
    enable_segmentation=True, # 포즈 검출과 인물 마스크를 생성할지 여부 결정. 배경 제거 등의 용도로 사용
    smooth_segmentation=True, # 인물 마스크의 경계를 부드럽게 처리
    min_detection_confidence=0.5, # 사람 검출 최소 신뢰도 임계값
    min_tracking_confidence=0.5 # 포즈 추적의 신뢰도 임계값
)

# 그리기 함수 초기화
mp_drawing = mp.solutions.drawing_utils

# 웹캠 캡처
cap = cv2.VideoCapture(0)

while True:
    # 이미지 읽기
    ret, image = cap.read()
    if not ret:
        break

    # 이미지 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 포즈 추정
    results = pose.process(image)

    # 추정된 포즈 정보 확인
    if results.pose_landmarks:

        # 연결된 랜드마크 시각화
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

    # 세그멘테이션 마스크 추출 및 배경 제거
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.segmentation_mask is not None:
        # 세그멘테이션 마스크를 이미지 크기에 맞게 조정
        segmentation_mask = results.segmentation_mask
        segmentation_mask = (segmentation_mask > 0.1)  # 임계값 적용
        segmentation_mask = segmentation_mask.astype(float)
        
        # 세그멘테이션 마스크를 3채널로 복제하여 컬러 이미지와 동일한 차원으로 만듦
        mask_three_channel = cv2.merge([segmentation_mask, segmentation_mask, segmentation_mask])
        
        # 마스크를 적용하여 배경을 제거한 이미지 생성
        foreground = cv2.multiply(mask_three_channel, image.astype(float))
        
        cv2.imshow('Foreground', foreground.astype('uint8'))

    # 이미지 출력
    cv2.imshow('MediaPipe Pose', image)

    # 키 입력 처리
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### 2. 손가락 포즈 검출 <br>
바위: 모든 손가락이 접혀 있음 <br>
가위: 엄지를 제외한 두 손가락(검지와 중지)만 펴져 있음 <br>
보: 모든 손가락이 펴져 있음 <br>

In [5]:
!pip install Pillow # 한글 출력을 위한 라이브러리 설치

In [6]:
from PIL import ImageFont, ImageDraw, Image
import numpy as np
# 한글을 출력하기 위한 함수
def draw_text(img, text, position, font_size, font_color):
    font_path = "C:/Windows/Fonts/gulim.ttc"  # Windows에서 Gulim 폰트 경로

    # opencv 이미지를 PIL이미지로 변환
    img_pil = Image.fromarray(img)

    # PIL Draw 객체 생성
    draw = ImageDraw.Draw(img_pil)

    # 폰트 스타일 지정
    font = ImageFont.truetype(font_path, font_size)

    # PIL 이미지에 텍스트 입력
    draw.text(position, text, font=font, fill=font_color)
    return np.array(img_pil) # 최종 numpy array 로 이미지 형태 반환

In [7]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

#mp_drawing = cv2.flip(mp_drawing, 1)

# 손가락의 개폐 상태를 기반으로 '가위', '바위', '보' 포즈를 판별하는 함수
def get_hand_pose(hand_landmarks):
    # 손가락 끝과 PIP(손가락 중간 관절) 사이의 거리를 비교하여 손가락이 펴져 있는지 확인
    open_fingers = []
    for i in [4, 8, 12, 16, 20]:  # 엄지부터 새끼손가락까지의 끝 랜드마크 인덱스
        tip = hand_landmarks.landmark[i]  # 손가락 끝
        pip = hand_landmarks.landmark[i - 2]  # 손가락 중간 관절

        #if handedness.classification[0].label == "Right":  # 오른손인 경우 판별하기
        #    if i == 4:  # 엄지손가락인 경우
        #        open_fingers.append(tip.x < pip.x)
                
        # 엄지는 x좌표를, 나머지 손가락은 y좌표를 사용하여 개폐 상태 확인 - 오른손 기준
        if i == 4:  # 엄지손가락인 경우
            open_fingers.append(tip.x > pip.x)
        else:
            open_fingers.append(tip.y < pip.y)
    
    # '가위', '바위', '보' 판별
    if open_fingers.count(True) == 5:
        return "보"
    elif open_fingers.count(True) == 2 and open_fingers[1]:
        return "가위"
    elif open_fingers.count(True) == 0:
        return "바위"
    else:
        return "모름"

cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크 그리기
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2))
            
            # 포즈 판별 및 출력
            pose = get_hand_pose(hand_landmarks)
            
            # 한글 출력을 위한 작업
            image = draw_text(image, pose, (10, 50), 30, (255, 255, 255))

    cv2.imshow('Hand Pose', image)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()